### Imports

In [ ]:
import autograd.numpy as np
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np
import time
import tqdm as tqdm

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_style('white')

import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm.notebook as tqdm

### Getting Test Datasets and Gradient Descent

In [ ]:
def get_dataset(name):
    images, labels, label_names = None, None, list(map(str, range(10)))
    if name == 'ones_and_zeros':
        mnist = tfds.image_classification.MNIST()
        mnist.download_and_prepare()
        data = tfds.as_numpy(mnist.as_dataset(split='train', as_supervised=True).batch(5000))
        images, labels = next(iter(data))
        images, labels = images[labels <= 1][:, :, :, 0].astype(float) / 128 - 1., labels[labels <= 1].astype(float)
    if name == 'mnist':
        mnist = tfds.image_classification.MNIST()
        mnist.download_and_prepare()
        data = tfds.as_numpy(mnist.as_dataset(split='train', as_supervised=True).batch(5000))
        images, labels = next(iter(data))
        images, labels = images[:, :, :, 0].astype(float) / 128 - 1., labels.astype(int)
    elif name == 'horses_and_humans':
        hh = tfds.image_classification.HorsesOrHumans()
        hh.download_and_prepare()
        data = tfds.as_numpy(hh.as_dataset(split='train', as_supervised=True).map(lambda x, y: (tf.image.resize(
        tf.image.rgb_to_grayscale(x), (48, 48)), y)
        ).batch(100000))
        images, labels = next(iter(data))
        images, labels = images[labels <= 1][:, :, :, 0].astype(float) / 128 - 1., labels[labels <= 1].astype(float)
        label_names = ['horse', 'human']
    elif name == 'cats_and_dogs':
        cd = tfds.image_classification.CatsVsDogs()
        cd.download_and_prepare()
        data = tfds.as_numpy(cd.as_dataset(split='train', as_supervised=True).map(lambda x, y: (tf.image.resize(
        tf.image.rgb_to_grayscale(x), (64, 64)), y)
        ).batch(100000))
        images, labels = next(iter(data))
        images, labels = images[labels <= 1][:, :, :, 0].astype(float) / 128 - 1., labels[labels <= 1].astype(float)
        label_names = ['cat', 'dog']
    elif name == 'iris':
        import sklearn.datasets as datasets
        dataset = datasets.load_iris()
        features = dataset['data']
        targets = dataset['target']
        feature_names = dataset['feature_names']
        target_names = dataset['target_names']
        images, labels = features, targets 
        label_names = target_names

    f, subplots = plt.subplots(8, 8, figsize=(20, 20))
    i = 0
    for row in subplots:
        for subplot in row:
            subplot.imshow(images[i], cmap='gray')
            subplot.axis('off')
            i += 1
    plt.show()
    return images, labels, label_names

def gradient_descent(model, X, y, lr=1e-6, steps=2500, image_shape=None, watch=True):
    image_shape = image_shape if model.weights.ndim == 1 else None
    dims = np.array(image_shape).prod()

    losses = []
    progress = tqdm.trange(steps)
    if watch:
        from IPython import display
        f, ax = plt.subplots(X.shape[1] // dims, 3, figsize=(15,8))
        ax = np.atleast_2d(ax)

    for i in progress:
        loss, g = model.nll_and_grad(X, y)
        model.weights = model.weights - lr * g
        accuracy = model.accuracy(X, y)

        losses.append(loss)
        progress.set_description('Loss %.2f, accuracy: %.2f' % (loss, accuracy))

        # Plotting code
        if watch:
            [a.cla() for a in ax.flatten()]
            [a.axis('off') for a in ax.flatten()[1:]]
            display.clear_output(wait =True)
            
            ax[0, 0].plot(losses)
            if not (image_shape is None):
                ax[0, 1].imshow(model.weights[:dims].reshape(image_shape))
                ax[0, 2].imshow(g[:dims].reshape(image_shape))
                ax[0, 1].set_title('Loss: %.3f, accuracy: %.3f' % (loss, accuracy))

                for j in range(1, ax.shape[0]):
                    ax[j, 1].imshow(model.weights[(dims * j):(dims * (j+1))].reshape(image_shape))
                    ax[j, 2].imshow(g[(dims * j):(dims * (j+1))].reshape(image_shape))
                    ax[j, 0].imshow((X[0, (dims * j):(dims * (j+1))].reshape(image_shape)) )

            display.display(f)
            time.sleep(0.001)
        
    return losses



### Logistic Regression Module

In [ ]:
def linear_function(X, w):
    # Returns a linear function of X (and adds bias)
    X = np.pad(X, ((0,0), (0,1)), constant_values=1.)
    return np.dot(X, w)

def sigmoid(x):
    # Computes the sigmoid function
    return 1 / (1 + np.exp(-x))

class LogisticRegression:
    def __init__(self, dims):
        '''
        Args:
            dims (int): d, the dimension of each input
        '''
        self.weights = np.zeros((dims + 1,))

    def predict(self, X):
        '''
        Predict labels given a set of inputs.

        Args:
            X (array): An N x d matrix of observations.
        Returns:
            pred (int array): A length N array of predictions in {0, 1}
        '''
        return (linear_function(X, self.weights) > 0).astype(int)    
    
    def predict_probability(self, X):
        '''
        Predict the probability of each class given a set of inputs

        Args:
            X (array): An N x d matrix of observations.
        Returns:
            probs (array): A length N vector of predicted class probabilities
        '''
        return sigmoid(linear_function(X, self.weights))

    def accuracy(self, X, y):
        '''
        Compute the accuracy of the model's predictions on a dataset

        Args:
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
        Returns:
            acc (float): The accuracy of the classifier
        '''
        return np.mean(self.predict(X) == y)    
    
    def nll(self, X, y):
        '''
        Compute the negative log-likelihood loss.

        Args:
            X (array): An N x d matrix of observations.
            y (int array): A length N vector of labels.
        Returns:
            nll (float): The NLL loss
        '''
        xw = linear_function(X, self.weights)
        py = sigmoid((2 * y - 1) * xw)
        return -np.sum(np.log(py))    
    
    def nll_gradient(self, X, y):
        '''
        Compute the gradient of the negative log-likelihood loss.

        Args:
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
        Returns:
            grad (array): A length (d + 1) vector with the gradient
        '''
        xw = linear_function(X, self.weights)
        py = sigmoid((2 * y - 1) * xw)
        grad = ((1 - py) * (2 * y - 1)).reshape((-1, 1)) * np.pad(X, [(0,0), (0,1)], constant_values=1.)
        return -np.sum(grad, axis=0)
    
    def nll_and_grad(self, X, y):
        '''
        Compute both the NLL and it's gradient

        Args:
            X (array): An N x d matrix of observations.
            y (array): A length N vector of labels.
        Returns:
            nll (float): The NLL loss
            grad (array): A length (d + 1) vector with the gradient
        '''
        return self.nll(X, y), self.nll_gradient(X, y)

### Testing logistic regression and feature transforms

In [ ]:
images, labels, label_names = get_dataset('horses_and_humans')

image_shape = images[0].shape                # Keep track of the original image shape
X = images.reshape((images.shape[0], -1))    # Reshape into an N x d matrix X
y = labels
print('Image shape: ', images.shape, ', X shape: ', X.shape)

model = LogisticRegression(X.shape[1])

In [ ]:

losses = gradient_descent(model, X, y, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

prediction = model.predict(X)
probabilities = model.predict_probability(X)

# Get the probability of the prediction [p(y=1) if prediction is 1 otherwise p(y=0)]
probability_of_prediction = np.where(prediction, probabilities, 1 - probabilities)

# Show an image and the corresponding prediction
plt.imshow(X[0].reshape(image_shape), cmap='gray')
print('Prediction: %s, probability: %.3f' % (label_names[prediction[0]], probability_of_prediction[0]))

In [ ]:
# Test and Train Split
def split_data(X, y):
    from sklearn.model_selection import train_test_split
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, test_size=0.3)

    return Xtrain, ytrain, Xtest, ytest

Xtrain, ytrain, Xtest, ytest = split_data(X, y)

In [ ]:
# Model Evaluation
train_model = LogisticRegression(Xtrain.shape[1])
losses = gradient_descent(train_model, Xtrain, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = train_model.accuracy(X = Xtrain, y = ytrain)
train_loss = train_model.nll(X = Xtrain, y = ytrain)

test_acc = train_model.accuracy(X = Xtest, y = ytest)
test_loss = train_model.nll(X = Xtest, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))

In [ ]:
# Quadratic Feature Transforms
Xtrain_quad = [np.append(Xtrain[0], [np.square(Xtrain[0])]).tolist()]
Xtest_quad = [np.append(Xtest[0], [np.square(Xtest[0])]).tolist()]

for i in range(1, len(Xtrain)):
   Xtrain_quad += [np.append(Xtrain[i], [np.square(Xtrain[i])]).tolist()]

for i in range(1, len(Xtest)):
   Xtest_quad += [np.append(Xtest[i], [np.square(Xtest[i])]).tolist()]

Xtrain_quad = np.asmatrix(Xtrain_quad)
Xtest_quad = np.asmatrix(Xtest_quad)

assert Xtrain_quad.shape == (Xtrain.shape[0], 2 * Xtrain.shape[1])

### Evaluating Transforms

In [ ]:
# Quadratic transforms
quad_model = LogisticRegression(Xtrain_quad.shape[1])
losses = gradient_descent(quad_model, Xtrain_quad, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = quad_model.accuracy(X = Xtrain_quad, y = ytrain)
train_loss = quad_model.nll(X = Xtrain_quad, y = ytrain)

test_acc = quad_model.accuracy(X = Xtest_quad, y = ytest)
test_loss = quad_model.nll(X = Xtest_quad, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))

In [ ]:
# Sin transforms
Xtrain_sin = [np.append(Xtrain[0], [np.sin(10 * Xtrain[0])]).tolist()]
Xtest_sin = [np.append(Xtest[0], [np.sin(10 * Xtest[0])]).tolist()]

for i in range(1, len(Xtrain)):
   Xtrain_sin += [np.append(Xtrain[i], [np.sin(10 * Xtrain[i])]).tolist()]

for i in range(1, len(Xtest)):
   Xtest_sin += [np.append(Xtest[i], [np.sin(10 * Xtest[i])]).tolist()]

Xtrain_sin = np.asmatrix(Xtrain_sin)
Xtest_sin = np.asmatrix(Xtest_sin)

assert Xtrain_sin.shape == (Xtrain.shape[0], 2 * Xtrain.shape[1])

# Training model
sin_model = LogisticRegression(Xtrain_sin.shape[1])
losses = gradient_descent(sin_model, Xtrain_sin, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = sin_model.accuracy(X = Xtrain_sin, y = ytrain)
train_loss = sin_model.nll(X = Xtrain_sin, y = ytrain)

test_acc = sin_model.accuracy(X = Xtest_sin, y = ytest)
test_loss = sin_model.nll(X = Xtest_sin, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))

### Multinomial Logistic Regression Module

In [ ]:
class MultinomialLogisticRegression(LogisticRegression):
    def __init__(self, classes, dims):
        '''
        Args:
            classes (int): C, the number of possible outputs
            dims (int): d, the dimension of each input
        '''
        self.classes = classes
        self.weights = np.zeros((classes, dims + 1,))

In [ ]:
def multiclass_predict(self, X):
    '''
    Predict labels given a set of inputs.

    Args:
        X (array): An N x d matrix of observations.
    Returns:
        pred (int array): A length N array of predictions in {0,...,(C-1)}
    '''
    W = self.weights
    pred_matrix = []
    pred = []

    for w in W:
        pred_matrix += [linear_function(X, w)]

    for i in range(0, len(pred_matrix[0])):
        pred += [np.argmax(np.squeeze(pred_matrix)[:, i])]

    return np.array(pred)

MultinomialLogisticRegression.predict = multiclass_predict

In [ ]:
def softmax(x):
    '''
    Apply the softmax function to a vector or matrix

    Args:
        X (array): An N x C matrix of transformed inputs (or a length C vector)
    Returns:
        probs (array):  An N x C matrix with the softmax function applied to each row
    ''' 

    if x.ndim == 1: return np.exp(x) / np.sum(np.exp(x))

    smax = []
    for row in x:
        smax += [np.exp(row) / np.sum(np.exp(row))]

    return np.asmatrix(smax)

In [ ]:
def nll(self, X, y):
    '''
    Compute the negative log-likelihood loss.

    Args:
        X (array): An N x d matrix of observations.
        y (int array): A length N vector of labels.
    Returns:
        nll (float): The NLL loss
    '''
    X = np.pad(X, ((0, 0), (0, 1)), constant_values = 1.)
    W = self.weights
    
    nll = 0
    for i in range(0, len(X)):
        term1 = np.dot(np.transpose(X[i]), W[y[i]])

        term2 = 0
        for j in range(0, len(W)):
            term2 += np.exp(np.dot(np.transpose(X[i]), W[j]))

        term2 = np.log(term2)
        nll += (term1 - term2)

    return -1 * nll

MultinomialLogisticRegression.nll = nll

In [ ]:
def nll_gradient_c(W, X, y, c):
    '''
    Compute the negative log-likelihood loss.

    Args:
        W (array): The C x d weight matrix.
        X (array): An N x d matrix of observations.
        y (int array): A length N vector of labels.
        c (int): The class to compute the gradient for
    Returns:
        grad (array): A length d vector representing the gradient with respect to w_c
    '''
    grad = 0
    X = np.pad(X, ((0, 0), (0, 1)), constant_values = 1.)

    for i in range(0, len(X)):
        grad += X[i] if y[i] == c else 0
        
        num = np.dot(X[i].T, np.exp(np.dot(X[i].T, W[c])))
        denom = np.sum(np.exp(np.dot(X[i], W.T)), axis = 0)
        term2 = num / denom

        grad -= term2
    
    return -1 * grad

In [ ]:
def nll_gradient(self, X, y):
    '''
    Compute the negative log-likelihood loss.

    Args:
        X (array): An N x d matrix of observations.
        y (int array): A length N vector of labels.
    Returns:
        grad (array): A C x d matrix representing the gradient with respect to W
    '''
    ## YOUR CODE HERE
    W = self.weights
    grad = []

    for c in range(0, len(W)):
        grad += [nll_gradient_c(W, X, y, c)]

    return np.array(grad)

MultinomialLogisticRegression.nll_gradient = nll_gradient

### Testing multinomial regression and feature transforms

In [ ]:
# MNIST dataset
images, labels, label_names = get_dataset('mnist')

image_shape = images[0].shape                # Keep track of the original image shape
X = images.reshape((images.shape[0], -1))    # Reshape into an N x d matrix X
y = labels
print('Image shape: ', images.shape, ', X shape: ', X.shape)

# Create the initial model
model = MultinomialLogisticRegression(classes=len(label_names), dims=X.shape[1])

In [ ]:
X, y = X[:500], y[:500]
Xtrain, ytrain, Xtest, ytest = split_data(X, y)

multi_train_model = MultinomialLogisticRegression(classes=len(label_names), dims=Xtrain.shape[1])
losses = gradient_descent(multi_train_model, Xtrain, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = multi_train_model.accuracy(X = Xtrain, y = ytrain)
train_loss = multi_train_model.nll(X = Xtrain, y = ytrain)

test_acc = multi_train_model.accuracy(X = Xtest, y = ytest)
test_loss = multi_train_model.nll(X = Xtest, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))

In [ ]:
# Visualizing learned weights
fig, ax = plt.subplots(10)
W = multi_train_model.weights

for i in range(0, len(W)):
    C = W[i][:-1].reshape(28, 28)
    ax[i].imshow(C)

plt.show()

In [ ]:
# Quadratic Transforms
Xtrain_quad = [np.append(Xtrain[0], [np.square(Xtrain[0])]).tolist()]
Xtest_quad = [np.append(Xtest[0], [np.square(Xtest[0])]).tolist()]

for i in range(1, len(Xtrain)):
   Xtrain_quad += [np.append(Xtrain[i], [np.square(Xtrain[i])]).tolist()]

for i in range(1, len(Xtest)):
   Xtest_quad += [np.append(Xtest[i], [np.square(Xtest[i])]).tolist()]

Xtrain_quad = np.asmatrix(Xtrain_quad)
Xtest_quad = np.asmatrix(Xtest_quad)

assert Xtrain_quad.shape == (Xtrain.shape[0], 2 * Xtrain.shape[1])

quad_multi_model = MultinomialLogisticRegression(classes=len(label_names), dims=Xtrain_quad.shape[1])
losses = gradient_descent(quad_multi_model, Xtrain_quad, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = quad_multi_model.accuracy(X = Xtrain_quad, y = ytrain)
train_loss = quad_multi_model.nll(X = Xtrain_quad, y = ytrain)

test_acc = quad_multi_model.accuracy(X = Xtest_quad, y = ytest)
test_loss = quad_multi_model.nll(X = Xtest_quad, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))

In [ ]:
# Sin feature transform
Xtrain_sin = [np.append(Xtrain[0], [np.sin(10 * Xtrain[0])]).tolist()]
Xtest_sin = [np.append(Xtest[0], [np.sin(10 * Xtest[0])]).tolist()]

for i in range(1, len(Xtrain)):
   Xtrain_sin += [np.append(Xtrain[i], [np.sin(10 * Xtrain[i])]).tolist()]

for i in range(1, len(Xtest)):
   Xtest_sin += [np.append(Xtest[i], [np.sin(10 * Xtest[i])]).tolist()]

Xtrain_sin = np.asmatrix(Xtrain_sin)
Xtest_sin = np.asmatrix(Xtest_sin)

assert Xtrain_sin.shape == (Xtrain.shape[0], 2 * Xtrain.shape[1])

# Training sin-transformed model
sin_multi_model = MultinomialLogisticRegression(classes=len(label_names), dims=Xtrain_sin.shape[1])
losses = gradient_descent(sin_multi_model, Xtrain_quad, ytrain, lr=1e-6, steps=2500, image_shape=image_shape, watch=False)

train_acc = sin_multi_model.accuracy(X = Xtrain_sin, y = ytrain)
train_loss = sin_multi_model.nll(X = Xtrain_sin, y = ytrain)

test_acc = sin_multi_model.accuracy(X = Xtest_sin, y = ytest)
test_loss = sin_multi_model.nll(X = Xtest_sin, y = ytest)

print('Training accuracy: %.3f, loss: %.3f' % (train_acc, train_loss))
print('Test accuracy: %.3f, loss: %.3f' % (test_acc, test_loss))